If the cell raises error of  "No module named 'chocoq'", please open a new terminal and excute
```bash
conda activate choco_cpu
```
or 
```bash
conda activate choco_gpu
```
enter into the `Choco-Q/` dir and excute
```bash
pip install .
```
reopen this notebook and rerun the cells.

## import the packages

In [ ]:
should_print = True # True for debug model and False for release mode.

from chocoq.model import LinearConstrainedBinaryOptimization as LcboModel
from chocoq.solvers.optimizers import CobylaOptimizer, AdamOptimizer
from chocoq.solvers.qiskit import (
    PenaltySolver, CyclicSolver, HeaSolver, ChocoSolver, 
    AerGpuProvider, AerProvider, FakeBrisbaneProvider, FakeKyivProvider, FakeTorinoProvider, DdsimProvider,
)

## customize the constrained problem

In [ ]:
# ----------------------- model -----------------------
m = LcboModel()
x = m.addVars(5, name="x") ## add five binary variables with prefix name x
m.setObjective((x[0] + x[1])* x[3] + x[2], "max") ## set the maximum objective with (x[0]+x[1])x[3] + x[2]
# m.addConstr(x[0] + x[1] + x[2] == 2)
# m.addConstr(x[0] + x[1] == 1)

m.addConstr(x[0] + x[1] - x[2] == 0) #add linear constraints
m.addConstr(x[2] + x[3] - x[4] == 1)

print(m.lin_constr_mtx) #print linear constraints matrix
print(m) # print the information of model.

[[ 1  1 -1  0  0  0]
 [ 0  0  1  1 -1  1]]
m:
variables:
x_0 (type: binary)   x_1 (type: binary)   x_2 (type: binary)   x_3 (type: binary)   x_4 (type: binary)

obj:
max 1 * x_0 * x_3 + 1 * x_1 * x_3 + 1 * x_2

s.t.:
1 * x_0 + 1 * x_1 + -1 * x_2 == 0
1 * x_2 + 1 * x_3 + -1 * x_4 == 1

penalty_lambda:
32767




The exact solution is

In [3]:
optimize = m.optimize()
print(f"optimize_cost: {optimize}\n\n")

Restricted license - for non-production use only - expires 2025-11-24
optimize_cost: (2.0, {'x_0': 0.0, 'x_1': 1.0, 'x_2': 1.0, 'x_3': 1.0, 'x_4': 1.0})




Solve the optimization problem by Choco-Q.

In [ ]:
# sovler ----------------------------------------------
opt = CobylaOptimizer(max_iter=200) # set the optimizer COBYLA, the max iteration is 200.
# gpu = AerGpuProvider()
aer = DdsimProvider()
solver = ChocoSolver(
    prb_model=m,  # the problem model
    optimizer=opt,  # the optimizer for parameters updating.
    provider=aer,  # quantum circuit simulator.
    num_layers=1,
)
print(solver.circuit_analyze(['depth', 'width', 'culled_depth', 'num_one_qubit_gates'])) 
## the circuit analyze results



best_cost: 2.0
best_solution_case: [0.0, 1.0, 1.0, 1.0, 1.0]

[127, 12, 62, 110]
iteration 10, result: 1.138671875
iteration 20, result: 1.099609375
iteration 30, result: 1.0849609375
[4.107  0.7682]

[0, 0, 0, 1, 0]: 0 ~ 0.0947265625
[1, 0, 1, 0, 0]: 1 ~ 0.685546875
[1, 0, 1, 1, 1]: 2 ~ 0.2197265625

best_solution_probs: 22.0
in_constraint_probs: 100.0
ARG: 0.4374999978
mean_cost: 1.1

([[0, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1]], [0.0947265625, 0.685546875, 0.2197265625], 32)
[21.97265625, 100.0, 0.43749999781250004, 32]


In [ ]:
result = solver.solve() # solving with input solver

print(result)

In [ ]:
eval = solver.evaluation() # evaluation of the solving results
print(eval) 